# The Ambulance Problem

In this problem, we are about to find the best route in which the ambulance can take all the patients to the nearest hospital availabe.

## Libraries

In this project, `time` library is used to measure time of each solution.
<br>
`Orderedset` is imported to implement BFS solver for this problem.(Odreded set is a set that keeps its items in order as well)
    

In [69]:
# !pip3 install ordered_set

import time
from ordered_set import OrderedSet

## States(or Nodes)
We model each state as an instance of Node class, in which we keep the location of the ambulance, all the patients, hospitals and walls of the map as well.
<br>
Each state is modeled using a Node. Then by using the function `find_next_nodes()`, each state will find the next states it can step into using an action. 
<br>
Directions are considered as our **actions**.(*Up*, *Down*, *Left*, *Right*) Moving the ambulance from one location to another is made by these actions.
<br>
<br>
<br>
We also need to rewrite the `__eq__` and `__hash__` functions due to out implementation.
<br>
<br>
First, we change the `__eq__` function cause the importance of states (that make them different as well) is their ambulance location, patients, and hospitals. In order to pay attention to these features and not anything else, the `__eq__` function has to change.
<br>
Second, as we are using `OrderedSet` ,we need to change the `__hash__` function in the way that we can keep it in sets, so we hash each instance to make it compatible with tuples.

In [70]:
class Node:
    def __init__(self, position, walls, patients, hospitals, g = 0):
        self.position = position
        self.walls = walls
        self.patients = patients
        self.hospitals = hospitals
        self.g = g
        self.h = 0
        self.f = 0
        self.parent = None
    
    def calc_h(self, method):
        for patient in self.patients: 
            temp = 100000000
            for hospital in self.hospitals:
                if method == 'h1':
                    dist = abs(hospital[0][0]-patient[0]) + abs(hospital[0][1] - patient[1])
                else:
                    dist = (abs(hospital[0][0]-patient[0])**2 + abs(hospital[0][1] - patient[1])**2) ** 0.5
                if  dist < temp and hospital[1] > 0:
                    temp = dist
            self.h += temp

    def find_next_nodes(self):
        next_legal_nodes = []
        legal_moves = [[0, 1], [0, -1], [1, 0], [-1, 0]]
        for move in legal_moves:
            new_position = (self.position[0] + move[0], self.position[1] + move[1])
            if new_position in self.walls:
                continue
            elif new_position in self.patients:
                if new_position[0] + move[0] < 0 or new_position[1] + move[1] < 0:
                    continue
                    
                new_patient_position = (new_position[0] + move[0], new_position[1] + move[1])
                if new_patient_position in self.patients or new_patient_position in self.walls:
                    continue
            
                founded = 0
                for hospital in self.hospitals:
                    if new_patient_position == hospital[0]:
                        new_patients = self.patients.copy()
                        new_hospitals = self.hospitals.copy()
                        if hospital[1] > 0:
                            new_patients.discard(new_position)
                            new_hospitals.discard(hospital)
                            new_hospitals.add((hospital[0], hospital[1]-1))
                        else:
                            new_patients.discard(new_position)
                            new_patients.add((new_position[0]+move[0], new_position[1]+move[1]))
                        next_legal_nodes.append(Node(new_position, self.walls, new_patients, new_hospitals, self.g+1))
                        founded = 1
                        break
                if not founded:
                    new_patients = self.patients.copy()
                    new_patients.discard(new_position)
                    new_patients.add((new_position[0]+move[0], new_position[1]+move[1]))
                    next_legal_nodes.append(Node(new_position, self.walls, new_patients, self.hospitals.copy(), self.g+1))
            else:
                next_legal_nodes.append(Node(new_position, self.walls, self.patients.copy(), self.hospitals.copy(), self.g+1))
        return next_legal_nodes
    
    def find_path(self):
        path = []
        current = self
        while current.parent:
            path.append(current)
            current = current.parent
        path.append(current)
        return path[::-1]

    
    def __eq__(self, new_node):
        return self.hospitals == new_node.hospitals and self.patients == new_node.patients and \
            self.position == new_node.position

    def __hash__(self):
        return hash((tuple(self.position), tuple(self.walls), tuple(self.patients), tuple(self.hospitals)))
    

**Initial state** is the first state that is inspired from the given map, having initial location of *ambulance*, *walls*, *patients*, and *hospitals*.
For finding the initial state(or start state), we tend to use a function called `find_startstate` explained below:
<br>
In this function by getting the testcase name as an input, we find the initial state. To accomplish this, first we need to read the given file using `open()` function. 
<br>
The data given as testcases, are 3 *.txt* files in which different maps are being kept. In all of these maps, '#' is considered as *wall*, 'P' is considered as *patient*, 'A' as *ambulance*, and numbers are *hospitals* with the capacity of that number.
<br>
According to our knowledge of given maps, we find the location of ambulance, patients, walls, and hospitals(plus their capacity) and collect them in sets. The reason for using sets is that they are fast, and will not keep repeated data.
<br>
Finally, these collected data would be given to a Node's constructor and make the first satate in the problem.


In [71]:
def find_startstate(file_name):
    with open(file_name, 'r') as f:
        map = list(f)
    map = np.array(map[0:])
    walls = set()
    patients = set()
    hospitals = set()
    position = np.array([0, 0])
    
    for i in range(map.shape[0]):
        for j in range(len(map[i])):
            if map[i][j] == '#':
                walls.add((i, j))
            elif map[i][j] == 'A':
                position = (i, j)
            elif map[i][j] == 'P':
                patients.add((i, j))
            elif map[i][j] == ' ' or map[i][j] == '\n':
                continue
            else:
                hospitals.add(((i, j), int(map[i][j])))
    return Node(position, walls, patients, hospitals)

def print_path(path):
    for item in path:
        print(item.position, "->", end=" ")
    print("END")
    
def print_result(all_states, new_states, depth, total_time, method):
    print(method, " takes %s seconds to find the optimal solution for " % total_time, test_name)
    print("Depth: ", depth)
    print("Number of all states: ", all_states)
    print("Number of unique states: ", new_states)


## Implementations:
The goal of this problem is to find a state in which all of the patients are in hospitals. To accomplish this, we define the **goal state** as *a state in which there is no patient left outside of hospitals*.
<br>
<br>
There are three ways to solve this Search Problem that are implemented below.
<br><br>

### 1. BFS 
One of the ways to find the optimal solution to this problem is to implement BFS.
<br>
BFS is a traversing algorithm which starts traversing from root(here, initial state) and traverse the graph layerwise which leads to exploring all the neighbor nodes at the present depth. It then moves towards the neighbour nodes at the next depth level. Neighbor nodes are the nodes(os states) that are directly accessible from the parent node.(first, root has no parent and it is the parent of all its neighbor)
<br>
<br>
This method is implemented below. To keep track of all nodes in order(their depth), an `OrderedSet` data structure is being used. Using this structure, Nodes are being kept in order, so we can pick them in order and add their childs(in the next depth level) to the set and also repeated states will not be explored.

In [72]:
def BFS(start_state):
    num_all_states = 0
    num_new_states = 0
    state_set = OrderedSet([start_state])
    index = 0
    while True:
        new_states = state_set[index].find_next_nodes()
        for new_state in new_states:
            num_all_states += 1
            new_state.parent = state_set[index]
            if len(new_state.patients) == 0:
                path = new_state.find_path()
                num_new_states = index+1
                return num_all_states, num_new_states, path
            
        
        state_set.update(new_states)
        index += 1
    num_new_states = index
    node = state_set[index]
    path = node.find_path()

    return num_all_states, num_new_states, path


In [73]:
test_name = 'test1.txt'
start_state = find_startstate(test_name)
start_time = time.time()
num_all_states, num_new_states, path = BFS(start_state)
total_time = time.time() - start_time
print_result(num_all_states, num_new_states, len(path)-1, total_time, "BFS")
print_path(path)

BFS  takes 0.010878324508666992 seconds to find the optimal solution for  test1.txt
Depth:  11
Number of all states:  962
Number of unique states:  341
(1, 3) -> (2, 3) -> (3, 3) -> (4, 3) -> (4, 4) -> (5, 4) -> (5, 3) -> (5, 2) -> (4, 2) -> (3, 2) -> (3, 1) -> (4, 1) -> END


### 2. IDS 
Iterative Deepening search or more specifically Iterative Deepening Depth-First Search is a graph search algorithm in which a depth-limited version of DFS(called DLS) is run repeatedly with increasing depth limits until the goal is found. IDS is an optimal optimal solution just like BFS, but uses much less memory according to its use of DFS algorithm.
<br>
`Depth-first search` (DFS) is an algorithm for traversing graph data structures. This algorithm starts traversing from the root node and explores as far as possible along each branch before backtracking. Then it steps back until it can find another way to explore.

In [74]:
def IDS(root: Node, maximum_depth: int=100):
    num_all_states = 1
    num_new_states = 1
    for depth in range(0, maximum_depth):
        visited = dict()
        founded, new_num_alls, new_num_news = DLS(root, visited, depth)
        num_all_states += new_num_alls
        if founded:
            num_new_states = new_num_news
            path = founded.find_path()
            return path, num_all_states, num_new_states
        
def DLS(root, visited, num):
    num_all_states = 0
    num_new_states = 0
    visited[hash(frozenset([root]))] = num

    if len(root.patients) == 0:
        return root, num_all_states, num_new_states
    if num <= 0:
        return None, num_all_states, num_new_states

    next_states = root.find_next_nodes()
    for new_state in next_states:
        new_state.parent = root
        validity = None
        if hash(frozenset([new_state])) in visited:
            validity = visited[hash(frozenset([new_state]))]
        
        num_all_states += 1
        if not validity:
            num_new_states += 1
        if validity is None or validity < (num-1):
            founded, new_num_alls, new_num_news = DLS(new_state, visited, num-1)
            num_all_states += new_num_alls
            num_new_states += new_num_news
            if founded:
                return founded, num_all_states, num_new_states
        else:
            continue

    return None, num_all_states, num_new_states

In [75]:
start_state = find_startstate('test3.txt')
start_time = time.time()
path, num_all_states, num_new_states = IDS(start_state)
total_time = time.time() - start_time
print_result(num_all_states, num_new_states, len(path)-1, total_time, "IDS")
print_path(path)

IDS  takes 11.772437334060669 seconds to find the optimal solution for  test1.txt
Depth:  39
Number of all states:  2428789
Number of unique states:  59560
(2, 8) -> (2, 7) -> (1, 7) -> (1, 6) -> (1, 7) -> (2, 7) -> (3, 7) -> (4, 7) -> (5, 7) -> (5, 6) -> (5, 5) -> (4, 5) -> (4, 6) -> (5, 6) -> (5, 7) -> (4, 7) -> (3, 7) -> (2, 7) -> (2, 8) -> (1, 8) -> (1, 7) -> (1, 6) -> (1, 7) -> (2, 7) -> (3, 7) -> (4, 7) -> (4, 6) -> (4, 5) -> (4, 4) -> (4, 3) -> (5, 3) -> (5, 2) -> (4, 2) -> (3, 2) -> (2, 2) -> (2, 1) -> (1, 1) -> (1, 2) -> (1, 3) -> (1, 4) -> END


## 3. A* 
A* Search algorithm is one of the best algorithms used in graph traversals. It uses the steps token before and the estimation of future to make an estimation(called heuristic) for its next step.
<br>
<br>
First, we define a frontier, in which we keep all the states that can be reached from now. Secondly, we define a measure called f, that is:
$$ Node.f = Node.g + Node.h$$
in which g is number of steps taken till now(distance) and h is the heuristic chosen for this problem.
<br>
We should note that our heuristic should be admissible and consistent. Otherwise it would not give us the optimal solution. To make sure that the answer is optimal, we check admissibility for every heuristic we suggest. Moreover, as we avoid having repeated states, it would not be necessary to check consistency.
Admissible choices are heuristics that have the following condition:
$$ h(n) \leq h^*(n) $$
where $h^*(n)$ is the true cost to reach the goal state.
<br>
<br>
For implementing this, we use two heuristics explained below:
<br>
<br>
The heuristic is the sum of minimum distances between patients and hospitals. In each state, the minimum distance needed to be taken to accomplish the goal, is to take every patient to his nearest hospital available. So this heuristic is les or equal to the actual distance(cost) and therefore, it is admissible due to its condition.
$$ h(n) \leq h^*(n)$$

This heuristic can be implemented in two different ways. One with **Manhattan Distance** and the other with **Euclidean distance**.
#### 3.1. Manhattan Distance 
*Manhattan distance* or *L1 distance* is a distance metric between two points in a N dimensional vector space. It is the sum of the lengths of the projections of the line segment between the points onto the coordinate axes. In simple terms, it is the sum of absolute difference between the measures in all dimensions of two points.
$$L_1(x, y) = ||x-y||_1 = \sum_{i=1}^{n}|x_i - y_i| $$

#### 3.2. Euclidean Distance
*Euclidean Distance* or *L2 distance* between two points in either the plane or n-dimensional space measures the length of a segment connecting the two points. It is the most obvious way of representing distance between two points.
$$L_2(x, y) = ||x-y||_2 = \sum_{i=1}^{n}|x_i - y_i|^2 $$

<br>
<br>
Both heuristics are admissible and therefore, both of them give us optimal. But since a patient can be moved along the four cardinal diections and not diagonally, the **Manhattan Distance** would be more close to the real cost. As a result, **L1 Distance** is a better choice. 

In [76]:
def A_star(start, method):

    current = start
    closed = set()
    frontier = dict()
    num_all_states = 0
    num_new_states = 0
    
    frontier[hash(frozenset([current]))] = current
    
    while len(frontier) != 0:
        temp = 10000000
        for key,value in frontier.items():
            if value.f < temp:
                temp = value.f
                current = value
                
        if len(current.patients) == 0:
            path = current.find_path()
            num_new_states = len(closed)
            return path, num_all_states, num_new_states

        del frontier[hash(frozenset([current]))]
        closed.add(current)
        
        new_states = Node.find_next_nodes(current)
        num_all_states += len(new_states)
        for node in new_states:
            if node in closed:
                continue
                
            node.calc_h(method)
            node.f = node.h + node.g
            if hash(frozenset([node])) in frontier:
                prev_node = frontier[hash(frozenset([node]))]
                
                if node.g < prev_node.g:
                    prev_node.g = node.g
                    prev_node.parent = current
            else:
                node.parent = current
                frontier[hash(frozenset([node]))] = node
    
    num_new_states = len(closed)
    return None, num_all_states, num_new_states

In [94]:
test_name ='test3.txt'
start_state = find_startstate(test_name)
start_time = time.time()
path , num_all_states, num_new_states = A_star(start_state, 'h2')
total_time = time.time() - start_time
print_result(num_all_states, num_new_states, len(path)-1, total_time, "A*")
print_path(path)

A*  takes 1.1937353610992432 seconds to find the optimal solution for  test3.txt
Depth:  39
Number of all states:  37072
Number of unique states:  13467
(2, 8) -> (2, 7) -> (1, 7) -> (1, 6) -> (1, 7) -> (2, 7) -> (3, 7) -> (4, 7) -> (5, 7) -> (5, 6) -> (5, 5) -> (4, 5) -> (4, 6) -> (5, 6) -> (5, 7) -> (4, 7) -> (3, 7) -> (2, 7) -> (2, 8) -> (1, 8) -> (1, 7) -> (1, 6) -> (1, 7) -> (2, 7) -> (3, 7) -> (4, 7) -> (4, 6) -> (4, 5) -> (4, 4) -> (4, 3) -> (5, 3) -> (5, 2) -> (4, 2) -> (3, 2) -> (2, 2) -> (2, 1) -> (1, 1) -> (1, 2) -> (1, 3) -> (1, 4) -> END


# Final Results

| Method | Distance(Depth) | Number of all states | Number of unique states | time
| --- | --- | --- | --- | --- |
| BFS | 11 | 962 | 341 | 0.1125 |
| IDS | 11 | 3907 | 544 | 0.035 |
| $A^*$(First heuristic) | 11 | 433 | 152 | 0.0106 |
| $A^*$(Second heuristic) | 11 | 468 | 164 | 0.0104 |
<caption  style="text-align:center">Result for testing 'test1.txt'</caption>


| Method | Distance(Depth) | Number of all states | Number of unique states | time
| --- | --- | --- | --- | --- |
| BFS | 27 | 26208 | 10465 | 0.2679 |
| IDS | 27 | 531425 | 16630 | 2.37 |
| $A^*$(First heuristic) | 27 | 12509 | 4879 | 0.2868 |
| $A^*$(Second heuristic) | 27 | 13809 | 5402 | 0.3765 |
<caption  style="text-align:center">Result for testing 'test2.txt'</caption>

| Method | Distance(Depth) | Number of all states | Number of unique states | time
| --- | --- | --- | --- | --- |
| BFS | 39 | 86039 | 31287 | 0.7279 |
| IDS | 39 | 2428789 | 59560 | 10.91 |
| $A^*$(First heuristic) | 39 | 31782 | 11535 | 0.9341 |
| $A^*$(Second heuristic) | 39 | 37072 | 13467 | 1.239 |
<caption  style="text-align:center">Result for testing 'test3.txt'</caption>

## Comparison 

 Different properties of these algorithms are shown below:
<br>
<br>

| Method | complete | optimal | time | space |
| --- | --- | --- | --- | --- |
| BFS | yes| yes| O$(4^{depth})$ | O$(4^{depth})$ |
| IDS | yes | yes |O$(4^{depth})$ | O$(4depth)$ | 
| $A^*$ | yes | yes | $\leq exponential$ | $exponential$ |
<caption  style="text-align:center">Properties of different algorithms</caption>

<br><br>
In the context of search:
<br>
* A complete algorithm is one that guarantees that if a path to the goal exists, the algorithm will reach the goal.

* An optimal algorithm is one that guarantees that it would give you the best answer possible for the problem.

 #### Advantage and Disadvantages of these algorithms
<br>
<br>
* It is obvious that *IDS* algorithm uses much less space as its space is in linear order but other algorithms tend to use exponential space. On the other hand, it should be noticed that in our implementation of *IDS*, we are trying to decrease the time by keeping track of visited states. It denies the fact that *IDS* uses much less space. Reality speaking, *IDS* algorithm takes too much time to show the result(much more than *BFS* and $A^*$)

$$ O(IDS) = (depth+1)4^0 + depth\times 4^1 + (depth-1)4^2 + ... + 4^{depth} = O(4^{depth}) $$ 

<br>
<br>

* It can be seen that the only algorithm using much less time (not exponential) is $A^*$ algorithm.
<br>
<br>

* *BFS* algorithm is fast, but memoty intensive. So it can not be used for large amount of data.